<a href="https://colab.research.google.com/github/nabin-nath/major-project-2/blob/electricity-consumption/Final_Time_domain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
#imports
import pandas as pd
import numpy as np

df0 = pd.read_csv('/content/drive/MyDrive/Major Project - II/0_interpolate_n.csv')
df1 = pd.read_csv('/content/drive/MyDrive/Major Project - II/1_interpolate_n.csv')

<ipython-input-3-19ce6e226e03>:5: DtypeWarning: Columns (0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263

In [ ]:
df0 = df0.iloc[:,2:]
df1 = df1.iloc[:3579,2:]
df0 = df0.drop(df0.index[0])
df1 = df1.drop(df1.index[0])

df0 = df0.fillna(0)
df0 = df0.astype('float64')
df1 = df1.fillna(0)
df1 = df1.astype('float64')

from sklearn.utils import resample
import pandas as pd
import numpy as np

# Randomly select the rows
n_rows=len(df0)-len(df1)
df2 = df0.sample(n=n_rows)

# Define a random multiplication factor between 0 and 0.8
mult_factor = np.random.uniform(low=0.2, high=0.8, size=(n_rows, df0.shape[1]))

# Multiply each column of the selected rows with the multiplication factor
df_modified = df2 * mult_factor
df_modified['1'] = 1.0

# Add the modified rows to a new dataframe
df1 = df1.append(df_modified)
# df1 = resample(df1,
#              replace=True,
#              n_samples=len(df0),
#              random_state=42)
             
df = pd.concat([df0, df1], ignore_index=True)

<ipython-input-4-e96897acb447>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df_modified)


In [ ]:
# Importing required libraries
from sklearn.model_selection import train_test_split
from scipy.interpolate import pchip_interpolate

# dataset shuffling
df = df.sample(frac=1).reset_index(drop=True)

# split the dataset into training, validation, and test datasets
trainvalX, testX, trainvaly, testy = train_test_split(df.iloc[:, 1:], df.iloc[:, :1], test_size=0.2, random_state=42)
trainX, valX, trainy, valy = train_test_split(trainvalX, trainvaly, test_size=0.2, random_state=42)

# Extract the statistical features from the data
train_std_dev = np.std(trainX, axis=1)
train_mean = np.mean(trainX, axis=1)
train_median = np.median(trainX, axis=1)
train_datamean = np.mean(trainX, axis=1)
train_cfpmean = np.mean(trainX, axis=1)

val_std_dev = np.std(valX, axis=1)
val_mean = np.mean(valX, axis=1)
val_median = np.median(valX, axis=1)
val_datamean = np.mean(valX, axis=1)
val_cfpmean = np.mean(valX, axis=1)

test_std_dev = np.std(testX, axis=1)
test_mean = np.mean(testX, axis=1)
test_median = np.median(testX, axis=1)
test_datamean = np.mean(testX, axis=1)
test_cfpmean = np.mean(testX, axis=1)

# Add time domain features
train_stdsProb = np.sum(np.abs(trainX - train_mean.values.reshape(-1, 1)), axis=1) / trainX.shape[1]
val_stdsProb = np.sum(np.abs(valX - val_mean.values.reshape(-1, 1)), axis=1) / valX.shape[1]
test_stdsProb = np.sum(np.abs(testX - test_mean.values.reshape(-1, 1)), axis=1) / testX.shape[1]

train_stdsMean = train_std_dev / train_mean
val_stdsMean = val_std_dev / val_mean
test_stdsMean = test_std_dev / test_mean

train_outsprob = np.sum((trainX - train_mean.values.reshape(-1, 1)) ** 2, axis=1) / trainX.shape[1]
val_outsprob = np.sum((valX - val_mean.values.reshape(-1, 1)) ** 2, axis=1) / valX.shape[1]
test_outsprob = np.sum((testX - test_mean.values.reshape(-1, 1)) ** 2, axis=1) / testX.shape[1]

# train_outsmean = np.abs(train_median - train_mean.values.reshape(-1, 1))
# val_outsmean = np.abs(val_median - val_mean.values.reshape(-1, 1))
# test_outsmean = np.abs(test_median - test_mean.values.reshape(-1, 1))

train_outsDev = train_std_dev / train_median
val_outsDev = val_std_dev / val_median
test_outsDev = test_std_dev / test_median

# # Create pchip interpolant
# train_interpolant = pchip_interpolate(np.arange(train_X.shape[1]), train_X.T)
# val_interpolant = pchip_interpolate(np.arange(val_X.shape[1]), val_X.T)
# test
trainFeatX = np.stack([train_std_dev, train_mean, train_median, train_datamean,  train_cfpmean, train_stdsProb, train_stdsMean, train_outsprob, train_outsDev], axis=1)
testFeatX = np.stack([test_std_dev, test_mean, test_median, test_datamean, test_cfpmean, test_stdsProb, test_stdsMean, test_outsprob, test_outsDev], axis=1)
valFeatX = np.stack([val_std_dev, val_mean, val_median, val_datamean, val_cfpmean, val_stdsProb, val_stdsMean, val_outsprob, val_outsDev], axis=1)

In [ ]:
valy = np.array(valy)
testy = np.array(testy)
trainy = np.array(trainy)

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split

# # Assuming your pandas DataFrame is named 'df'
# # Extract the features (consumption values) from the DataFrame
# features = df.iloc[:, 1:].values

# # Extract the target variable (class labels) from the DataFrame
# target = df.iloc[:, 0].values

# # Split the data into train and test sets with a 0.2 ratio
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# # Further split the train set into train and validation sets with a 0.2 ratio
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# # Create DataFrames for train, test, and validation sets
# train_data = pd.DataFrame(X_train, columns=df.columns[1:])
# train_data['class'] = y_train

# test_data = pd.DataFrame(X_test, columns=df.columns[1:])
# test_data['class'] = y_test

# val_data = pd.DataFrame(X_val, columns=df.columns[1:])
# val_data['class'] = y_val

# # Create a new DataFrame to store the time domain features for the training set
# train_features = pd.DataFrame()

# # Iterate over each column in the training set
# for col in train_data.columns[:-1]:  # Exclude the last column (class label)
#     # Compute the rolling mean, standard deviation, minimum, and maximum
#     rolling_mean = train_data[col].rolling(window=10).mean()
#     rolling_std = train_data[col].rolling(window=10).std()
#     rolling_min = train_data[col].rolling(window=10).min()
#     rolling_max = train_data[col].rolling(window=10).max()

#     # Compute percentiles (25th, 50th, and 75th)
#     # rolling_percentiles25 = train_data[col].rolling(window=10).quantile(0.25)
#     # rolling_percentiles5 = train_data[col].rolling(window=10).quantile(0.5)
#     # rolling_percentiles75 = train_data[col].rolling(window=10).quantile(0.75)

#     # Concatenate the time domain features for the current column
#     # col_features = pd.concat([rolling_mean, rolling_std, rolling_min, rolling_max, rolling_percentiles25, rolling_percentiles5, rolling_percentiles75], axis=1)

#     col_features = pd.concat([rolling_mean, rolling_std, rolling_min, rolling_max], axis=1)
#     # Rename the columns with appropriate labels
#     # col_features.columns = [
#     #     f'{col}_mean', f'{col}_std', f'{col}_min', f'{col}_max', f'{col}_25th_percentile',
#     #     f'{col}_50th_percentile', f'{col}_75th_percentile'
#     # ]
#     col_features.columns = [
#         f'{col}_mean', f'{col}_std', f'{col}_min', f'{col}_max'
#     ]

#     # Append the time domain features to the new DataFrame
#     train_features = pd.concat([train_features, col_features], axis=1)

# # Display the extracted time domain features for the training set
# print(train_features)

In [ ]:
# train_features

In [ ]:
# train_cfpmean

In [ ]:
# print(len(valX))
# print(len(trainX))
# print(len(testX))

In [ ]:
#input scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(trainX)
scaled_train = scaler.transform(trainX)
scaled_test = scaler.transform(testX)
scaled_val = scaler.transform(valX)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
# from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras_tuner.tuners import RandomSearch

In [ ]:
import keras_tuner

In [ ]:
# class EeHyperModel(keras_tuner.HyperModel):
#     def build(self, hp):
#         n_timesteps, n_features, n_outputs = trainX.shape[1], 1, trainy.shape[1]
#         model = Sequential()
#         model.add(LSTM(hp.Int('units_LSTM', 80, 220, step=30), input_shape=(n_timesteps,n_features)))
#         model.add(Dropout(hp.Choice('dropout', [0.2, 0.5])))

#         for i in range(hp.Int('layers', 1, 3)):
#             model.add(Dense(
#                 units=hp.Int('units_' + str(i), 30, 200, step=30),
#                 activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])
#             ))

#         model.add(Dense(n_outputs, activation='sigmoid'))
#         model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(
#             hp.Choice('learning_rate', values=[1e-2, 1e-4])),
#                       metrics=['accuracy'])

#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         history = model.fit(
#             *args,
#             validation_split=0.2,
#             batch_size=hp.Int("batch_size", 32, 264, step=32),
#             epochs=50,
#             callbacks=[tf.keras.callbacks.EarlyStopping(
#                 monitor='val_loss',
#                 patience=5,
#                 restore_best_weights=True
#             )],
#             **kwargs,
#         )
#         return history


In [ ]:
# class EeHyperModel(keras_tuner.HyperModel):
#     def build(self, hp):
#       n_timesteps, n_features, n_outputs = trainX.shape[1], 1, trainy.shape[1]
#       model = Sequential()
#       model.add(LSTM(hp.Int('units_LSTM', 80, 220, step=30), input_shape=(n_timesteps,n_features)))
#       model.add(Dropout(hp.Choice('dropout', [0.2, 0.5])))

#       for i in range(hp.Int('layers', 1, 3)):
#         model.add(Dense(
#           units=hp.Int('units_' + str(i), 30, 200, step=30),
#           activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])    
#         ))

#       model.add(Dense(n_outputs, activation='sigmoid'))
#       model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(
#           hp.Choice('learning_rate',
#                     values=[1e-2, 1e-4])),
#                     metrics=['accuracy'])

#       return model
    
#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             batch_size=hp.Int("batch_size", 32, 264, step=32),
#             **kwargs,
#         )

In [ ]:
# tuner = RandomSearch(
#     EeHyperModel(),
#     objective='val_accuracy',
#     max_trials=5,
#     executions_per_trial=1,
#     directory='my_dir',
#     project_name='rnn_lstm'
# )

In [ ]:
# tuner.search_space_summary()

In [ ]:
# Will stop training if the "val_loss" hasn't improved in 3 epochs.
# tuner.search(trainX, trainy, epochs=2,
#              validation_data=(valX,valy), 
#              callbacks=[keras.callbacks.EarlyStopping('val_loss', patience=3)])

# tuner.search(trainX, trainy, epochs=5,
#              validation_data=(valX,valy))

In [ ]:
# tuner.results_summary()

In [ ]:
# import the necessary packages
from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate

In [ ]:
# train model
verbose, epochs, batch_size = 1, 15, 64
n_timesteps, n_features, n_outputs = trainX.shape[1], trainFeatX.shape[1], trainy.shape[1]

series_in = Input(shape=(n_timesteps, 1))
other_in = Input(shape=(n_features,)) # not a timeseries just a vector

# An example graph
lstm_out = LSTM(128)(series_in)
merged = concatenate([lstm_out, other_in])
out = Dense(n_outputs, activation='sigmoid')(merged)
model = Model([series_in, other_in], out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# model = Sequential()
# model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
# model.add(Dropout(0.2))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(n_outputs))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model fitting
model.fit(
	[trainX, trainFeatX], trainy,
	validation_data=([valX, valFeatX], valy),
	epochs=10, batch_size=64)
# model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
# evaluation
# _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)

Epoch 1/10
734/734 [==============================] - 37s 48ms/step - loss: 501517385728.0000 - accuracy: 0.6277 - val_loss: 301606797312.0000 - val_accuracy: 0.6234
Epoch 2/10
734/734 [==============================] - 32s 43ms/step - loss: 1282483027968.0000 - accuracy: 0.6229 - val_loss: 1158082854912.0000 - val_accuracy: 0.6238
Epoch 3/10
734/734 [==============================] - 32s 43ms/step - loss: 10673792745472.0000 - accuracy: 0.6414 - val_loss: 261999869952.0000 - val_accuracy: 0.6590
Epoch 4/10
734/734 [==============================] - 34s 46ms/step - loss: 421566283776.0000 - accuracy: 0.7927 - val_loss: 339463700480.0000 - val_accuracy: 0.8472
Epoch 5/10
218/734 [=======>......................] - ETA: 19s - loss: 4911454486528.0000 - accuracy: 0.8112

KeyboardInterrupt: ignored

In [ ]:
# Evaluate the model on the test data
# my_model = tuner.get_best_models(num_models=1)[0]
# print(my_model[0])
test_loss, test_accuracy = model.evaluate([testX, testFeatX], testy)

# # Print the test accuracy
print("Test accuracy:", test_accuracy)

459/459 [==============================] - 7s 16ms/step - loss: 11644046409728.0000 - accuracy: 0.8505
Test accuracy: 0.8505316376686096


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score

y_pred_test = model.predict([testX, testFeatX])
y_true_test = testy
# y_true_test = np.array(y_true_test['1'])

y_pred_val = model.predict([valX, valFeatX])
y_true_val = valy
# y_true_val = np.array(y_true_val['1'])

roc_auc = roc_auc_score(y_true_test, y_pred_test)
print('Model: ROC AUC=%.3f' % (roc_auc))

367/367 [==============================] - 5s 14ms/step
Model: ROC AUC=0.865


In [ ]:
y_pred_val = np.array(y_pred_val > 0.5)
y_pred_val = y_pred_val.flatten()

# validation %
precision_recall_fscore_support(y_true_val, y_pred_val, average=None, labels=[True, False])

(array([0.82566204, 0.87541713]),
 array([0.88629442, 0.81022649]),
 array([0.85490452, 0.84156122]),
 array([5910, 5828]))

In [ ]:
y_pred_test = np.array(y_pred_test > 0.5)
y_pred_test = y_pred_test.flatten()

# test %
precision_recall_fscore_support(y_true_test, y_pred_test, average=None, labels=[True, False])

(array([0.82453656, 0.87977984]),
 array([0.88527989, 0.81672717]),
 array([0.85382923, 0.84708179]),
 array([7235, 7437]))